## Exploratory Data Analysis

Reference: https://wrds-www.wharton.upenn.edu/pages/wrds-research/applications/institutional-ownership-research/thomson-reuters-13f-database-and-classification-institutional-investors-python/

In [8]:
import pandas as pd
import time

import numpy as np
import wrds
import pull_wrds

import config
from pathlib import Path

OUTPUT_DIR = config.OUTPUT_DIR
DATA_DIR = config.DATA_DIR
WRDS_USERNAME = config.WRDS_USERNAME


In [9]:
print(DATA_DIR)

C:/Users/062/GITHUB/DSFinalProject/data


In [12]:
comp = pull_wrds.load_13f(data_dir=DATA_DIR)
crsp = pull_wrds.load_Mutual_Fund(data_dir=DATA_DIR)

In [13]:
display(comp)

,fdate,mgrno,mgrname,typecode,cusip,shares,prc,shrout1
0,1980-03-31,44175.0,HAWAIIAN TRUST LIMITED,1.0,None,NaN,NaN,NaN
1,1980-03-31,44225.0,HEIZER CORPORATION,5.0,None,NaN,NaN,NaN
2,1980-03-31,90415.0,VARIABLE ANNUITY INS CO,2.0,None,NaN,NaN,NaN
3,1980-03-31,14560.0,CENTRAL NATL/RICHMOND,1.0,02473510,41550.0,32.00,28.0
4,1980-03-31,14560.0,CENTRAL NATL/RICHMOND,1.0,00282410,13000.0,36.63,60.0
...,...,...,...,...,...,...,...,...
144884,2017-12-31,82810.0,TEACHERS ADVR INC,2.0,98986X10,23139.0,3.83,24.0
144885,2017-12-31,84900.0,CITIGROUP INC,5.0,98986X10,685.0,3.83,24.0
144886,2017-12-31,90457.0,"VANGUARD GROUP, INC.",5.0,98986X10,599151.0,3.83,24.0
144887,2017-12-31,67780.0,PANAGORA ASSET MANAGEMENT INC.,5.0,98986M10,13800.0,11.07,33.0


In [14]:
display(crsp)

,fdate,mgrco
0,1999-03-31,AAL CAPITAL MANAGEMENT
1,1999-03-31,ASEA BROWN BOVERI INVT MGMT A
2,1999-03-31,ABN AMRO LUXEMBOURG
3,1999-03-31,"ACADIAN ASSET MANAGEMENT, INC"
4,1999-03-31,"ACADEMY CAPITAL MGMT, INC."
...,...,...
153576,2017-12-31,ADULA CAPITAL AG
153577,2017-12-31,WPS ADVR LIMITED (BERMUDA)
153578,2017-12-31,"URSUS-3 CAPITAL, AV, S.A"
153579,2017-12-31,VERDE ASSET MANAGEMENT S.A.


In [15]:
comp.rename(columns={'fdate': 'date'}, inplace=True)
comp['date'] = pd.to_datetime(comp['date'])
grouped = comp.groupby(['mgrno', 'date'])

In [19]:
comp_sorted = comp.sort_values(by=['mgrno', 'date'])

In [21]:
display(comp_sorted)

,date,mgrno,mgrname,typecode,cusip,shares,prc,shrout1
353453,1998-12-31,110.0,A R ASSET MANAGEMENT INC,4.0,03783310,23100.0,40.94,135.0
355777,1998-12-31,110.0,A R ASSET MANAGEMENT INC,4.0,00282410,3500.0,49.00,1518.0
360280,1998-12-31,110.0,A R ASSET MANAGEMENT INC,4.0,03948310,8792.0,17.13,593.0
361748,1998-12-31,110.0,A R ASSET MANAGEMENT INC,4.0,00792410,2641.0,122.25,580.0
361925,1998-12-31,110.0,A R ASSET MANAGEMENT INC,4.0,01747510,1078.0,46.63,113.0
...,...,...,...,...,...,...,...,...
47090,1993-06-30,95120.0,ZWEIG TOTAL RETURN ADVS.,3.0,88284810,77300.0,45.75,220.0
47106,1993-06-30,95120.0,ZWEIG TOTAL RETURN ADVS.,3.0,90478450,33600.0,105.75,160.0
47123,1993-06-30,95120.0,ZWEIG TOTAL RETURN ADVS.,3.0,91191010,28500.0,47.50,108.0
47179,1993-06-30,95120.0,ZWEIG TOTAL RETURN ADVS.,3.0,93114210,147800.0,26.25,2300.0


In [22]:
nunique_dates = grouped['date'].nunique().sum()
nunique_mgrno = comp['mgrno'].nunique()

print(f"Number of unique dates: {nunique_dates}")
print(f"Number of unique 'mgrno': {nunique_mgrno}")


Number of unique dates: 306243
Number of unique 'mgrno': 8954


In [23]:
nunique_dates = comp_sorted['date'].nunique()
nunique_mgrno = comp_sorted['mgrno'].nunique()

print(f"Number of unique dates: {nunique_dates}")
print(f"Number of unique 'mgrno': {nunique_mgrno}")

Number of unique dates: 152
Number of unique 'mgrno': 8954


In [24]:
(2018-1980)*4

152

Quarterly reports

In [25]:
nunique_mgrco = crsp['mgrco'].nunique()
print(f"Number of unique 'mgrco': {nunique_mgrco}")

Number of unique 'mgrco': 11875


Need to match companies?

In [26]:
unique_typecodes = comp_sorted['typecode'].unique()
print(unique_typecodes)

[ 4.  5.  3.  2.  1. nan]


Split bins of years

In [27]:
bins = pd.to_datetime(['1980-01-01', '1985-01-01', '1990-01-01', '1995-01-01', 
                       '2000-01-01', '2005-01-01', '2010-01-01', '2015-01-01', '2018-01-01'])
indices = pd.cut(comp_sorted['date'], bins=bins, right=False, labels=None, retbins=False, include_lowest=True)
dataframes = {interval: df for interval, df in comp_sorted.groupby(indices)}

for interval, df in dataframes.items():
    filename = f"comp_sorted_{interval.left.strftime('%Y-%m-%d')}_{interval.right.strftime('%Y-%m-%d')}.csv"
    df.to_csv(filename, index=False)

C:\Users\062\AppData\Local\Temp\ipykernel_25772\536390498.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dataframes = {interval: df for interval, df in comp_sorted.groupby(indices)}


From our reference document,
```
ts = ts.rename(columns={1:'Bank',\
                        2:'Insurance',\
                        3:'Mutual Funds', \
                        4:'Investment Advisors',\
                        5:'Others'}).reset_index()
```

Pasting the whole relevant code:

In [30]:
df_1980_1984 = pd.read_csv('comp_sorted_1980-01-01_1985-01-01.csv')
df_unique_mgrno = df_1980_1984.drop_duplicates(subset='mgrno')
typecode_counts = df_unique_mgrno['typecode'].value_counts()

In [31]:
print(typecode_counts)

typecode
1.0    326
4.0    308
5.0    120
2.0     87
3.0     73
Name: count, dtype: int64


In [32]:
more_than_one_typecode = df_1980_1984[df_1980_1984.duplicated(subset=['mgrno', 'typecode'], keep=False)]
multi_typecode_mgrno = more_than_one_typecode['mgrno'].unique()

list(multi_typecode_mgrno)

[230.0,
 250.0,
 260.0,
 520.0,
 560.0,
 650.0,
 700.0,
 835.0,
 885.0,
 975.0,
 980.0,
 1170.0,
 1250.0,
 1300.0,
 1365.0,
 1380.0,
 1400.0,
 1790.0,
 1900.0,
 2010.0,
 2080.0,
 2200.0,
 2340.0,
 2470.0,
 2730.0,
 3510.0,
 3650.0,
 3700.0,
 3845.0,
 3880.0,
 3900.0,
 4200.0,
 4400.0,
 4675.0,
 4695.0,
 4710.0,
 4720.0,
 4730.0,
 4760.0,
 4810.0,
 4850.0,
 4900.0,
 5070.0,
 5145.0,
 5200.0,
 5460.0,
 5720.0,
 5800.0,
 5850.0,
 5900.0,
 5940.0,
 5960.0,
 5980.0,
 6000.0,
 6110.0,
 6500.0,
 6600.0,
 6700.0,
 6890.0,
 7020.0,
 7200.0,
 7345.0,
 7500.0,
 7600.0,
 7700.0,
 7800.0,
 7850.0,
 7900.0,
 8070.0,
 8100.0,
 8190.0,
 8225.0,
 8230.0,
 8235.0,
 8238.0,
 8240.0,
 8245.0,
 8250.0,
 8255.0,
 8350.0,
 8580.0,
 8650.0,
 8710.0,
 8750.0,
 8760.0,
 8765.0,
 8800.0,
 8840.0,
 9000.0,
 9100.0,
 9360.0,
 9480.0,
 9540.0,
 9550.0,
 9600.0,
 9750.0,
 9850.0,
 10000.0,
 10100.0,
 10200.0,
 10465.0,
 10515.0,
 10575.0,
 10600.0,
 10660.0,
 10750.0,
 10800.0,
 10835.0,
 10850.0,
 10900.0,
 11000.0

In [33]:
column_names = df_1980_1984.columns.tolist()

if 'mgrno' in column_names:
    multi_typecodes = df_1980_1984.groupby('mgrno')['typecode'].nunique()
    multi_typecode_mgrno = multi_typecodes[multi_typecodes > 1].index
    count_multi_typecode_mgrno = len(multi_typecode_mgrno)

count_multi_typecode_mgrno

59

In [28]:
##########################################
# Time Series of Number of Institutions  #
# by Manager Type                        #
# October 2018                           #
# Qingyi (Freda) Song Drechsler          #
##########################################

import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt

###################
# Connect to WRDS #
###################
conn=wrds.Connection()

# Read S34Type1 Dataset
# rdate = Holdings Report Date
# fdate = Vintage Date
s34type1 = conn.raw_sql("""
                    select fdate, rdate, typecode, mgrno 
                    from tfn.s34type1 
                    """, date_cols = ['fdate', 'rdate'])
s34type1[['typecode','mgrno']]=s34type1[['typecode','mgrno']].astype(int)
s34type1 = s34type1[s34type1['rdate']==s34type1['fdate']]

# Count the number of institutions by typecode
instcounts = s34type1.groupby(['rdate','typecode'])['mgrno'].count()\
.to_frame().reset_index()
instcounts=instcounts.rename(columns={'mgrno':'numero'})

# Reshape (transpose) the table 
# Report number by typecode per column
ts = instcounts.pivot(index='rdate', columns='typecode', values='numero')

# Rename the columns by label
ts = ts.rename(columns={1:'Bank',\
                        2:'Insurance',\
                        3:'Mutual Funds', \
                        4:'Investment Advisors',\
                        5:'Others'}).reset_index()

# Fill in 0 for missing observation
ts = ts.fillna(0)

# Calculate Total by adding up all categories
ts['Total']=ts['Bank']+ts['Insurance']+ts['Mutual Funds']+ts['Investment Advisors']+ts['Others']

# Year and Month information from rdate
ts['Month']=ts['rdate'].dt.month
ts['Year']=ts['rdate'].dt.year

ts=ts.set_index('rdate')

# Results at Year End
# See Table Output in the section section
ts_yend = ts[ts['Month']==12].drop(['Month'],axis=1)

# Plot the time series of the charts
ts = ts.drop(['Month', 'Year'], axis=1)
ts.plot(kind='line', title = 'Time Series of the Number of Institutions by Manager Type',\
        linewidth=4, figsize=(12,8), fontsize=15)

OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/14/e3q8)